In [2]:
%pwd

'd:\\Projects\\Medical-Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data from PDF file
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [7]:
extract_data=load_pdf_file(data='Data/')

In [237]:
#Splitting the data into smaller text chunks
def text_splitter(extract_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extract_data)
    return text_chunks

In [238]:
text_chunks = text_splitter(extract_data)
print("Length of Text chunks: ", len(text_chunks))

Length of Text chunks:  39994


In [55]:
from langchain.embeddings import HuggingFaceEmbeddings




In [46]:
#Download the embeddings from HuggingFace
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2'
        
    )
    return embeddings



In [62]:
embeddings = download_hugging_face_embeddings()

d:\Projects\Medical-Chatbot\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [63]:
query_results = embeddings.embed_query("Hello World")
print("Length", len(query_results))

Length 384


In [239]:
from dotenv import load_dotenv
load_dotenv()
PINE_CONE_API_KEY = os.environ.get("PINECONE_API_KEY")


In [240]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc =Pinecone(api_key=PINE_CONE_API_KEY)

index_name= "medibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1")
        )

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '73ebe90ff13da716119e003f915cad9c', 'date': 'Sat, 05 Apr 2025 06:23:03 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [241]:
#Emberd each text chunk into the Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,index_name=index_name, embedding=embeddings)

KeyboardInterrupt: 

In [181]:
#load existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name, embedding=embeddings)


In [182]:
docsearch

In [183]:
retriever = docsearch.as_retriever(search_type="similarity" ,search_kwargs={"k": 3})

In [184]:
retrieved_docs = retriever.invoke("What is epilepsy?")

In [ ]:
retrieved_docs

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI


load_dotenv()


gemini_api_key = os.environ.get('GEMINI_API_KEY')
print(gemini_api_key)

if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY not found in .env file.")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", max_output_tokens=2048,google_api_key=gemini_api_key)

result = llm.invoke("Explain the benefits of using Langchain with Gemini 2.0 Flash.")
print(result.content)

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API Key not found. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API Key not found. Please pass a valid API key."
]

In [215]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt =( 
    "you are an assistant that answers questions about medical conditions. "
    "You have access to a large database of medical information. " 
    "use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, just say you don't know. Use three sentences maximum and keep the answer concise. "
    "If the question is not related to medical conditions, just say you are not a medical assistant. " \
    "\n\n"
    "{context} "
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{query}"),
    ]
)

In [216]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [217]:
response = rag_chain.invoke({"input": "What is epilepsy?"})
print(response)


KeyError: "Input to ChatPromptTemplate is missing variables {'query'}.  Expected: ['context', 'query'] Received: ['input', 'context']\nNote: if you intended {query} to be part of the string and not a variable, please escape it with double curly braces like: '{{query}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "